# Modelo de Clasificación - Gastos Adicionales de Vivienda

## Objetivo
Predecir si un asegurado presentará siniestros en la cobertura de Gastos Adicionales de Vivienda (clasificación binaria).

## Pipeline
1. **Fase 1**: Preprocesamiento + GLM + LazyPredict
2. **Fase 2**: Optimización de top 5 modelos con GridSearchCV
3. **Fase 3**: Evaluación y selección del mejor modelo

**Dataset**: 7,999 registros | Desbalanceo: 94.8% no siniestrados vs 5.2% siniestrados

## 1. Imports y Configuración

In [19]:
# Manipulación de datos
import pandas as pd
import numpy as np

# Preprocesamiento
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score,
    roc_curve, precision_recall_curve, make_scorer
)

# LazyPredict
from lazypredict.Supervised import LazyClassifier

# Selección de variables
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
np.random.seed(42)

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. Carga y Preparación de Datos

In [20]:
# Cargar datos
df = pd.read_csv("../data/processed/adicionales_full.csv")

print(f"Dataset original: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
df.head()

Dataset original: (7999, 9)

Columnas: ['año_cursado', 'estudios_area', 'calif_promedio', '2_o_mas_inquilinos', 'distancia_al_campus', 'genero', 'extintor_incendios', 'Gastos_Adicionales_siniestros_num', 'Gastos_Adicionales_siniestros_monto']


,año_cursado,estudios_area,calif_promedio,2_o_mas_inquilinos,distancia_al_campus,genero,extintor_incendios,Gastos_Adicionales_siniestros_num,Gastos_Adicionales_siniestros_monto
0,3er año,Humanidades,3.01,No,0.00,Masculino,Si,0.00,0.00
1,3er año,Ciencias,1.52,No,0.00,Femenino,Si,0.00,0.00
2,3er año,Administracion,7.68,No,0.22,Femenino,No,0.00,0.00
3,4to año,Administracion,8.06,No,0.00,Masculino,No,0.00,0.00
4,2do año,Administracion,6.72,No,0.00,Femenino,No,0.00,0.00


In [21]:
# Eliminar columna de monto (no se usa en clasificación)
df = df.drop('Gastos_Adicionales_siniestros_monto', axis=1)

# Crear variable binaria objetivo: 0 si no siniestró, 1 si siniestró
df['siniestrado'] = (df['Gastos_Adicionales_siniestros_num'] > 0).astype(int)

# Eliminar columna original de número de siniestros
df = df.drop('Gastos_Adicionales_siniestros_num', axis=1)

print(f"\nDataset preparado: {df.shape}")
print(f"\nDistribución variable objetivo:")
print(df['siniestrado'].value_counts())
print(f"\nPorcentaje siniestrados: {df['siniestrado'].mean()*100:.2f}%")
print(f"Desbalanceo: {df['siniestrado'].value_counts()[0] / df['siniestrado'].value_counts()[1]:.1f}:1")


Dataset preparado: (7999, 8)

Distribución variable objetivo:
siniestrado
0    7583
1     416
Name: count, dtype: int64

Porcentaje siniestrados: 5.20%
Desbalanceo: 18.2:1


## 3. Análisis Exploratorio Rápido

In [22]:
# Identificar tipos de variables
numeric_features = ['calif_promedio', 'distancia_al_campus']
categorical_features = ['año_cursado', 'estudios_area', '2_o_mas_inquilinos', 'genero', 'extintor_incendios']

print("Variables numéricas:", numeric_features)
print("Variables categóricas:", categorical_features)
print(f"\nTotal features: {len(numeric_features) + len(categorical_features)}")

# Verificar valores faltantes
print(f"\nValores faltantes:\n{df.isnull().sum()}")

Variables numéricas: ['calif_promedio', 'distancia_al_campus']
Variables categóricas: ['año_cursado', 'estudios_area', '2_o_mas_inquilinos', 'genero', 'extintor_incendios']

Total features: 7

Valores faltantes:
año_cursado            0
estudios_area          0
calif_promedio         0
2_o_mas_inquilinos     0
distancia_al_campus    0
genero                 0
extintor_incendios     0
siniestrado            0
dtype: int64


In [23]:
# Estadísticas descriptivas por clase
print("=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS POR CLASE")
print("=" * 80)

for var in categorical_features:
    print(f"\n{var}:")
    tabla = pd.crosstab(df[var], df['siniestrado'], normalize='index') * 100
    print(tabla.round(2))

ESTADÍSTICAS DESCRIPTIVAS POR CLASE

año_cursado:
siniestrado     0    1
año_cursado           
1er año     94.14 5.86
2do año     95.21 4.79
3er año     94.88 5.12
4to año     94.59 5.41
posgrado    95.62 4.38

estudios_area:
siniestrado        0    1
estudios_area            
Administracion 94.58 5.42
Ciencias       94.99 5.01
Humanidades    95.41 4.59
Otro           94.22 5.78

2_o_mas_inquilinos:
siniestrado            0    1
2_o_mas_inquilinos           
No                 95.68 4.32
Si                 91.15 8.85

genero:
siniestrado      0    1
genero                 
Femenino     94.86 5.14
Masculino    94.50 5.50
No respuesta 95.98 4.02
Otro         95.86 4.14

extintor_incendios:
siniestrado            0    1
extintor_incendios           
No                 94.38 5.62
Si                 94.98 5.02


## 4. Preprocesamiento y Split de Datos

In [24]:
# Separar features y target
X = df.drop('siniestrado', axis=1)
y = df['siniestrado']

# Split train/test estratificado 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

print(f"Train set: {X_train.shape} | Siniestrados: {y_train.sum()} ({y_train.mean()*100:.2f}%)")
print(f"Test set:  {X_test.shape} | Siniestrados: {y_test.sum()} ({y_test.mean()*100:.2f}%)")

Train set: (6399, 7) | Siniestrados: 333 (5.20%)
Test set:  (1600, 7) | Siniestrados: 83 (5.19%)


In [25]:
# Pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ],
    remainder='drop'
)

# Ajustar y transformar datos
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Obtener nombres de features después de one-hot encoding
feature_names_num = numeric_features
feature_names_cat = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
feature_names = list(feature_names_num) + list(feature_names_cat)

# Convertir a DataFrame (mantener índices originales para alinear con y_train/y_test)
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names, index=X_train.index)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names, index=X_test.index)

print(f"\nFeatures después de preprocesamiento: {len(feature_names)}")
print(f"Shape X_train: {X_train_df.shape}")
print(f"Shape X_test: {X_test_df.shape}")


Features después de preprocesamiento: 14
Shape X_train: (6399, 14)
Shape X_test: (1600, 14)


## 5. Fase 1 - GLM con Selección Backward de Variables

In [26]:
# Modelo logístico completo para referencia
X_train_const = sm.add_constant(X_train_df)
logit_full = sm.Logit(y_train, X_train_const)
result_full = logit_full.fit(disp=0)

print("=" * 80)
print("MODELO LOGÍSTICO COMPLETO (todas las variables)")
print("=" * 80)
print(result_full.summary())

MODELO LOGÍSTICO COMPLETO (todas las variables)


                           Logit Regression Results                           
Dep. Variable:            siniestrado   No. Observations:                 6399
Model:                          Logit   Df Residuals:                     6384
Method:                           MLE   Df Model:                           14
Date:                Fri, 10 Oct 2025   Pseudo R-squ.:                 0.02224
Time:                        02:23:43   Log-Likelihood:                -1279.3
converged:                       True   LL-Null:                       -1308.4
Covariance Type:            nonrobust   LLR p-value:                 2.408e-07
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.7643      0.183    -15.132      0.000      -3.122      -2.406
calif_promedio               -0.0519      0.056     -0.921      0.357      -0.162     

In [27]:
# Selección Backward basada en p-valores
def backward_elimination(X, y, significance_level=0.05):
    """
    Realiza selección backward de variables basada en p-valores del GLM.
    """
    features = list(X.columns)
    
    while True:
        X_const = sm.add_constant(X[features])
        model = sm.Logit(y, X_const).fit(disp=0)
        p_values = model.pvalues.iloc[1:]
        max_p_value = p_values.max()
        
        if max_p_value <= significance_level:
            break
            
        feature_to_remove = p_values.idxmax()
        features.remove(feature_to_remove)
        print(f"Eliminando: {feature_to_remove} (p-valor: {max_p_value:.4f})")
    
    return features, model

print("=" * 80)
print("SELECCIÓN BACKWARD DE VARIABLES (α = 0.1)")
print("=" * 80)
print()

selected_features, logit_backward = backward_elimination(X_train_df, y_train, significance_level=0.1)

print()
print("=" * 80)
print(f"VARIABLES SELECCIONADAS: {len(selected_features)}")
print("=" * 80)
for feat in selected_features:
    print(f"  • {feat}")
print()
print(logit_backward.summary())

SELECCIÓN BACKWARD DE VARIABLES (α = 0.1)

Eliminando: genero_Otro (p-valor: 0.7660)
Eliminando: año_cursado_3er año (p-valor: 0.6920)
Eliminando: año_cursado_4to año (p-valor: 0.5401)
Eliminando: estudios_area_Otro (p-valor: 0.5434)
Eliminando: genero_Masculino (p-valor: 0.5134)
Eliminando: calif_promedio (p-valor: 0.3606)
Eliminando: año_cursado_posgrado (p-valor: 0.3071)
Eliminando: año_cursado_2do año (p-valor: 0.2712)
Eliminando: genero_No respuesta (p-valor: 0.1630)
Eliminando: distancia_al_campus (p-valor: 0.1063)

VARIABLES SELECCIONADAS: 4
  • estudios_area_Ciencias
  • estudios_area_Humanidades
  • 2_o_mas_inquilinos_Si
  • extintor_incendios_Si

                           Logit Regression Results                           
Dep. Variable:            siniestrado   No. Observations:                 6399
Model:                          Logit   Df Residuals:                     6394
Method:                           MLE   Df Model:                            4
Date:              

In [28]:
# Expandir variables categóricas completas
def expand_categorical_features(selected_features, all_feature_names, categorical_features):
    selected_cat_vars = set()
    for feat in selected_features:
        for cat_var in categorical_features:
            if feat.startswith(cat_var + '_'):
                selected_cat_vars.add(cat_var)
                break
    
    final_features = []
    for feat in selected_features:
        is_categorical = any(feat.startswith(cat_var + '_') for cat_var in categorical_features)
        if not is_categorical:
            final_features.append(feat)
    
    for feat in all_feature_names:
        for cat_var in selected_cat_vars:
            if feat.startswith(cat_var + '_') and feat not in final_features:
                final_features.append(feat)
                break
    
    return sorted(final_features)

final_selected_features = expand_categorical_features(selected_features, feature_names, categorical_features)

print("=" * 80)
print("FEATURES FINALES (con todos los niveles de categóricas)")
print("=" * 80)
print(f"\nTotal features: {len(final_selected_features)}")
print("\nFeatures seleccionadas:")
for feat in final_selected_features:
    indicator = "✓" if feat in selected_features else "+"
    print(f"  {indicator} {feat}")

X_train_selected = X_train_df[final_selected_features]
X_test_selected = X_test_df[final_selected_features]

print(f"\nShape final - Train: {X_train_selected.shape} | Test: {X_test_selected.shape}")

FEATURES FINALES (con todos los niveles de categóricas)

Total features: 5

Features seleccionadas:
  ✓ 2_o_mas_inquilinos_Si
  ✓ estudios_area_Ciencias
  ✓ estudios_area_Humanidades
  + estudios_area_Otro
  ✓ extintor_incendios_Si

Shape final - Train: (6399, 5) | Test: (1600, 5)


## 6. Fase 1 - LazyPredict para Identificar Mejores Algoritmos

In [29]:
# Ejecutar LazyPredict
print("=" * 80)
print("EJECUTANDO LAZYPREDICT")
print("=" * 80)
print(f"\nDataset: {X_train_selected.shape[0]} muestras, {X_train_selected.shape[1]} features")
print("Esto puede tomar varios minutos...\n")

clf = LazyClassifier(verbose=0, ignore_warnings=True, predictions=True, random_state=42)
models, predictions = clf.fit(X_train_selected, X_test_selected, y_train, y_test)

print("\n" + "=" * 80)
print("RESULTADOS DE LAZYPREDICT")
print("=" * 80)
print()
print(models)

EJECUTANDO LAZYPREDICT

Dataset: 6399 muestras, 5 features
Esto puede tomar varios minutos...



  0%|          | 0/32 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 333, number of negative: 6066
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000238 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15
[LightGBM] [Info] Number of data points in the train set: 6399, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.052039 -> initscore=-2.902312
[LightGBM] [Info] Start training from score -2.902312
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

---
## Exportación del Modelo Seleccionado

In [30]:
# Top 5 modelos + modelos adicionales solicitados
print("=" * 80)
print("TOP 5 MODELOS POR F1-SCORE")
print("=" * 80)
print()

top_5_models = models.nlargest(5, 'F1 Score')
print(top_5_models)

# Agregar modelos adicionales solicitados explícitamente
modelos_adicionales = ['NearestCentroid', 'LogisticRegression', 'XGBClassifier', 'RandomForestClassifier']
modelos_para_optimizar = list(set(list(top_5_models.index) + modelos_adicionales))

print("\n" + "=" * 80)
print("MODELOS SELECCIONADOS PARA FASE 2 (GridSearchCV)")
print("=" * 80)
print(f"\nTotal modelos a optimizar: {len(modelos_para_optimizar)}")
print("\nDe LazyPredict (Top 5):")
for i, model_name in enumerate(top_5_models.index, 1):
    f1 = top_5_models.loc[model_name, 'F1 Score']
    acc = top_5_models.loc[model_name, 'Accuracy']
    print(f"{i}. {model_name} - F1: {f1:.4f} | Accuracy: {acc:.4f}")

print("\nModelos adicionales solicitados:")
for model_name in modelos_adicionales:
    if model_name in models.index:
        f1 = models.loc[model_name, 'F1 Score']
        acc = models.loc[model_name, 'Accuracy']
        print(f"  • {model_name} - F1: {f1:.4f} | Accuracy: {acc:.4f}")
    else:
        print(f"  • {model_name} - (no evaluado en LazyPredict)")

TOP 5 MODELOS POR F1-SCORE

                        Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                    
AdaBoostClassifier          0.95               0.50     0.50      0.92   
BernoulliNB                 0.95               0.50     0.50      0.92   
CalibratedClassifierCV      0.95               0.50     0.50      0.92   
DecisionTreeClassifier      0.95               0.50     0.50      0.92   
DummyClassifier             0.95               0.50     0.50      0.92   

                        Time Taken  
Model                               
AdaBoostClassifier            0.13  
BernoulliNB                   0.01  
CalibratedClassifierCV        0.06  
DecisionTreeClassifier        0.01  
DummyClassifier               0.01  

MODELOS SELECCIONADOS PARA FASE 2 (GridSearchCV)

Total modelos a optimizar: 9

De LazyPredict (Top 5):
1. AdaBoostClassifier - F1: 0.9229 | Accuracy: 0.9481
2. BernoulliNB - F1: 0.9229 | Accur

---
## FASE 2: Optimización de Hiperparámetros con GridSearchCV

In [31]:
# Funciones de evaluación
def gini_coefficient(y_true, y_pred_proba):
    auc = roc_auc_score(y_true, y_pred_proba)
    return 2 * auc - 1

def find_optimal_threshold(y_true, y_pred_proba, metric='f1'):
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = []
    
    for threshold in thresholds:
        y_pred = (y_pred_proba >= threshold).astype(int)
        if metric == 'f1':
            score = f1_score(y_true, y_pred)
        elif metric == 'precision':
            score = precision_score(y_true, y_pred, zero_division=0)
        elif metric == 'recall':
            score = recall_score(y_true, y_pred)
        scores.append(score)
    
    optimal_idx = np.argmax(scores)
    return thresholds[optimal_idx], scores[optimal_idx]

print("✓ Funciones de evaluación definidas")

✓ Funciones de evaluación definidas


In [32]:
# Grillas de hiperparámetros con parámetros de suavizado/regularización
param_grids = {
    'AdaBoostClassifier': {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.5, 1.0],
        'algorithm': ['SAMME'],  # SAMME.R no está disponible en versiones recientes
        'estimator': [
            DecisionTreeClassifier(max_depth=1, random_state=42),  # Stumps
            DecisionTreeClassifier(max_depth=2, random_state=42),
            DecisionTreeClassifier(max_depth=3, random_state=42)
        ]
    },
    
    'BernoulliNB': {
        'alpha': [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],  # Suavizado de Laplace
        'binarize': [0.0, 0.3, 0.5, 0.7, None],
        'fit_prior': [True, False]
    },
    
    'CalibratedClassifierCV': {
        'method': ['sigmoid', 'isotonic'],
        'cv': [3, 5],
        'estimator': [
            LogisticRegression(C=0.1, penalty='l2', solver='liblinear', random_state=42, max_iter=1000),
            LogisticRegression(C=1.0, penalty='l2', solver='liblinear', random_state=42, max_iter=1000),
            DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, random_state=42)
        ]
    },
    
    'DecisionTreeClassifier': {
        'max_depth': [3, 5, 7, 10],
        'min_samples_split': [10, 20, 50],
        'min_samples_leaf': [5, 10, 20],
        'max_features': ['sqrt', 'log2'],
        'criterion': ['gini', 'entropy'],
        'class_weight': ['balanced', None],
        'ccp_alpha': [0.0, 0.01, 0.05]  # Poda de complejidad de costos
    },
    
    'DummyClassifier': {
        'strategy': ['stratified', 'most_frequent', 'prior']
    },
    
    # NUEVOS MODELOS
    'NearestCentroid': {
        'metric': ['euclidean', 'manhattan'],
        'shrink_threshold': [None, 0.1, 0.3, 0.5, 1.0, 2.0, 5.0]  # Regularización por encogimiento
    },
    
    'LogisticRegression': {
        'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],  # Inverso de fuerza de regularización
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear'],  # Compatible con l1 y l2
        'class_weight': ['balanced', None],
        'max_iter': [1000]
    },
    
    'XGBClassifier': {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [50, 100, 200],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0],
        'reg_alpha': [0, 0.1, 1.0],  # L1 regularization
        'reg_lambda': [1, 2, 5],  # L2 regularization
        'scale_pos_weight': [18.2],  # Ratio de desbalanceo (7583/416)
        'random_state': [42],
        'eval_metric': ['logloss']
    },
    
    'RandomForestClassifier': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7, 10, None],
        'min_samples_split': [10, 20, 50],
        'min_samples_leaf': [5, 10, 20],
        'max_features': ['sqrt', 'log2'],
        'class_weight': ['balanced', 'balanced_subsample', None],
        'ccp_alpha': [0.0, 0.01, 0.05],  # Poda de complejidad
        'random_state': [42]
    }
}

model_mapping = {
    'AdaBoostClassifier': AdaBoostClassifier(random_state=42),
    'BernoulliNB': BernoulliNB(),
    'CalibratedClassifierCV': CalibratedClassifierCV(),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
    'DummyClassifier': DummyClassifier(random_state=42),
    'NearestCentroid': NearestCentroid(),
    'LogisticRegression': LogisticRegression(random_state=42),
    'XGBClassifier': XGBClassifier(random_state=42, eval_metric='logloss'),
    'RandomForestClassifier': RandomForestClassifier(random_state=42)
}

print("✓ Grillas con parámetros de regularización/suavizado:")
print("  - AdaBoostClassifier: learning_rate + estimator (algoritmo SAMME)")
print("  - BernoulliNB: alpha (suavizado Laplace)")
print("  - CalibratedClassifierCV: estimator regularizado")
print("  - DecisionTreeClassifier: grilla reducida con parámetros de poda")
print("  - DummyClassifier: baseline")
print("  - NearestCentroid: shrink_threshold (regularización)")
print("  - LogisticRegression: C (regularización L1/L2) + class_weight")
print("  - XGBClassifier: reg_alpha/lambda + scale_pos_weight para desbalanceo")
print("  - RandomForestClassifier: poda + class_weight balanceado")

✓ Grillas con parámetros de regularización/suavizado:
  - AdaBoostClassifier: learning_rate + estimator (algoritmo SAMME)
  - BernoulliNB: alpha (suavizado Laplace)
  - CalibratedClassifierCV: estimator regularizado
  - DecisionTreeClassifier: grilla reducida con parámetros de poda
  - DummyClassifier: baseline
  - NearestCentroid: shrink_threshold (regularización)
  - LogisticRegression: C (regularización L1/L2) + class_weight
  - XGBClassifier: reg_alpha/lambda + scale_pos_weight para desbalanceo
  - RandomForestClassifier: poda + class_weight balanceado


In [33]:
# GridSearchCV
print("=" * 80)
print("GRID SEARCH CON VALIDACIÓN CRUZADA")
print("=" * 80)
print()

f1_scorer = make_scorer(f1_score)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

optimized_models = {}
grid_results = {}

for i, model_name in enumerate(modelos_para_optimizar, 1):
    print(f"\n{'='*80}")
    print(f"[{i}/{len(modelos_para_optimizar)}] Optimizando: {model_name}")
    print(f"{'='*80}")
    
    if model_name not in param_grids or model_name not in model_mapping:
        print(f"⚠ No hay grilla para {model_name}, saltando...")
        continue
    
    base_model = model_mapping[model_name]
    param_grid = param_grids[model_name]
    
    n_combinations = np.prod([len(v) if isinstance(v, list) else 1 for v in param_grid.values()])
    print(f"Combinaciones: {int(n_combinations)} | Total fits: {int(5 * n_combinations)}")
    
    try:
        grid_search = GridSearchCV(
            estimator=base_model,
            param_grid=param_grid,
            cv=cv,
            scoring=f1_scorer,
            n_jobs=-1,
            verbose=0
        )
        
        grid_search.fit(X_train_selected, y_train)
        
        optimized_models[model_name] = grid_search.best_estimator_
        grid_results[model_name] = {
            'best_params': grid_search.best_params_,
            'best_score_cv': grid_search.best_score_
        }
        
        print(f"✓ F1-Score CV: {grid_search.best_score_:.4f}")
        print(f"Mejores parámetros: {grid_search.best_params_}")
        
    except Exception as e:
        print(f"✗ Error: {str(e)}")

print(f"\n{'='*80}")
print(f"Completado: {len(optimized_models)}/{len(modelos_para_optimizar)} modelos")

GRID SEARCH CON VALIDACIÓN CRUZADA


[1/9] Optimizando: DecisionTreeClassifier
Combinaciones: 864 | Total fits: 4320
✓ F1-Score CV: 0.1199
Mejores parámetros: {'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 10}

[2/9] Optimizando: DummyClassifier
Combinaciones: 3 | Total fits: 15
✓ F1-Score CV: 0.0649
Mejores parámetros: {'strategy': 'stratified'}

[3/9] Optimizando: LogisticRegression
Combinaciones: 24 | Total fits: 120
✓ F1-Score CV: 0.1421
Mejores parámetros: {'C': 0.01, 'class_weight': 'balanced', 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear'}

[4/9] Optimizando: NearestCentroid
Combinaciones: 14 | Total fits: 70
✓ F1-Score CV: 0.1421
Mejores parámetros: {'metric': 'euclidean', 'shrink_threshold': 2.0}

[5/9] Optimizando: CalibratedClassifierCV
Combinaciones: 12 | Total fits: 60
✓ F1-Score CV: 0.0000
Mejores parámetros: {'cv': 3, 'estimator': LogisticRegression(C=0.1, 

In [34]:
# Evaluación en test set
print("=" * 80)
print("EVALUACIÓN EN TEST SET")
print("=" * 80)

test_results = []

for model_name, model in optimized_models.items():
    print(f"\n{'='*80}")
    print(f"Evaluando: {model_name}")
    print(f"{'='*80}")
    
    if hasattr(model, 'predict_proba'):
        y_pred_proba = model.predict_proba(X_test_selected)[:, 1]
    elif hasattr(model, 'decision_function'):
        y_pred_proba = model.decision_function(X_test_selected)
        y_pred_proba = (y_pred_proba - y_pred_proba.min()) / (y_pred_proba.max() - y_pred_proba.min())
    else:
        y_pred_proba = model.predict(X_test_selected).astype(float)
    
    optimal_threshold, _ = find_optimal_threshold(y_test, y_pred_proba)
    y_pred_optimal = (y_pred_proba >= optimal_threshold).astype(int)
    
    accuracy = accuracy_score(y_test, y_pred_optimal)
    precision = precision_score(y_test, y_pred_optimal, zero_division=0)
    recall = recall_score(y_test, y_pred_optimal, zero_division=0)
    f1 = f1_score(y_test, y_pred_optimal, zero_division=0)
    
    try:
        auc = roc_auc_score(y_test, y_pred_proba)
        gini = gini_coefficient(y_test, y_pred_proba)
    except:
        auc = gini = np.nan
    
    test_results.append({
        'Modelo': model_name,
        'F1_CV': grid_results[model_name]['best_score_cv'],
        'F1_Test': f1,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'AUC': auc,
        'Gini': gini,
        'Threshold': optimal_threshold
    })
    
    print(f"Threshold: {optimal_threshold:.3f}")
    print(f"F1 (CV): {grid_results[model_name]['best_score_cv']:.4f} | F1 (Test): {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f} | Precision: {precision:.4f} | Recall: {recall:.4f}")
    print(f"AUC: {auc:.4f} | Gini: {gini:.4f}")
    
    cm = confusion_matrix(y_test, y_pred_optimal)
    print(f"\nMatriz de Confusión:")
    print(f"  TN: {cm[0,0]:4d} | FP: {cm[0,1]:4d}")
    print(f"  FN: {cm[1,0]:4d} | TP: {cm[1,1]:4d}")

df_results = pd.DataFrame(test_results).sort_values('F1_Test', ascending=False)

print(f"\n{'='*80}")
print("RESUMEN DE RESULTADOS")
print(f"{'='*80}")
print()
print(df_results.to_string(index=False))

EVALUACIÓN EN TEST SET

Evaluando: DecisionTreeClassifier
Threshold: 0.560
F1 (CV): 0.1199 | F1 (Test): 0.1277
Accuracy: 0.7950 | Precision: 0.0819 | Recall: 0.2892
AUC: 0.5398 | Gini: 0.0796

Matriz de Confusión:
  TN: 1248 | FP:  269
  FN:   59 | TP:   24

Evaluando: DummyClassifier
Threshold: 0.100
F1 (CV): 0.0649 | F1 (Test): 0.0706
Accuracy: 0.9012 | Precision: 0.0690 | Recall: 0.0723
AUC: 0.5094 | Gini: 0.0189

Matriz de Confusión:
  TN: 1436 | FP:   81
  FN:   77 | TP:    6

Evaluando: LogisticRegression
Threshold: 0.510
F1 (CV): 0.1421 | F1 (Test): 0.1307
Accuracy: 0.7837 | Precision: 0.0825 | Recall: 0.3133
AUC: 0.5358 | Gini: 0.0715

Matriz de Confusión:
  TN: 1228 | FP:  289
  FN:   57 | TP:   26

Evaluando: NearestCentroid
Threshold: 0.470
F1 (CV): 0.1421 | F1 (Test): 0.1307
Accuracy: 0.7837 | Precision: 0.0825 | Recall: 0.3133
AUC: 0.5496 | Gini: 0.0992

Matriz de Confusión:
  TN: 1228 | FP:  289
  FN:   57 | TP:   26

Evaluando: CalibratedClassifierCV
Threshold: 0.100
F1 

---
## Exportación del Modelo Seleccionado

In [35]:
# Reentrenar DummyClassifier con TODO el dataset usando mejores hiperparámetros
print("=" * 80)
print("REENTRENAMIENTO CON DATASET COMPLETO")
print("=" * 80)
print()

# Combinar train y test
X_full = pd.concat([X_train_selected, X_test_selected])
y_full = pd.concat([y_train, y_test])

print(f"Dataset completo: {X_full.shape}")
print(f"Siniestrados totales: {y_full.sum()} ({y_full.mean()*100:.2f}%)")

# Obtener los mejores parámetros del DummyClassifier
best_params_dummy = grid_results['DummyClassifier']['best_params']
print(f"\nMejores parámetros del DummyClassifier:")
for param, value in best_params_dummy.items():
    print(f"  {param}: {value}")

# Crear y entrenar modelo con todos los datos
dummy_final = DummyClassifier(**best_params_dummy)
dummy_final.fit(X_full, y_full)

# Actualizar en el diccionario de modelos optimizados
optimized_models['DummyClassifier'] = dummy_final

print(f"\n✓ Modelo reentrenado con {len(X_full)} registros")
print(f"✓ Modelo actualizado en optimized_models")

REENTRENAMIENTO CON DATASET COMPLETO

Dataset completo: (7999, 5)
Siniestrados totales: 416 (5.20%)

Mejores parámetros del DummyClassifier:
  strategy: stratified

✓ Modelo reentrenado con 7999 registros
✓ Modelo actualizado en optimized_models


In [36]:
import pickle
import os

# Crear directorio models si no existe
os.makedirs('../models', exist_ok=True)

# Preparar objeto para exportar
modelo_export = {
    'modelo': optimized_models['DummyClassifier'],
    'preprocessor': preprocessor,
    'features_seleccionadas': final_selected_features,
    'threshold_optimo': df_results[df_results['Modelo'] == 'DummyClassifier']['Threshold'].values[0],
    'metricas': {
        'F1_CV': grid_results['DummyClassifier']['best_score_cv'],
        'F1_Test': df_results[df_results['Modelo'] == 'DummyClassifier']['F1_Test'].values[0],
        'Accuracy': df_results[df_results['Modelo'] == 'DummyClassifier']['Accuracy'].values[0],
        'Precision': df_results[df_results['Modelo'] == 'DummyClassifier']['Precision'].values[0],
        'Recall': df_results[df_results['Modelo'] == 'DummyClassifier']['Recall'].values[0],
        'AUC': df_results[df_results['Modelo'] == 'DummyClassifier']['AUC'].values[0],
        'Gini': df_results[df_results['Modelo'] == 'DummyClassifier']['Gini'].values[0]
    },
    'mejores_parametros': grid_results['DummyClassifier']['best_params']
}

# Exportar modelo
model_path = '../models/clasificacion_adicionales.pkl'
with open(model_path, 'wb') as f:
    pickle.dump(modelo_export, f)

print("=" * 80)
print("MODELO EXPORTADO EXITOSAMENTE")
print("=" * 80)
print(f"\nRuta: {model_path}")
print(f"\nModelo: DummyClassifier")
print(f"Mejores parámetros: {modelo_export['mejores_parametros']}")
print(f"\nMétricas en Test Set:")
print(f"  F1-Score:  {modelo_export['metricas']['F1_Test']:.4f}")
print(f"  Accuracy:  {modelo_export['metricas']['Accuracy']:.4f}")
print(f"  Precision: {modelo_export['metricas']['Precision']:.4f}")
print(f"  Recall:    {modelo_export['metricas']['Recall']:.4f}")
print(f"  AUC-ROC:   {modelo_export['metricas']['AUC']:.4f}")
print(f"  Gini:      {modelo_export['metricas']['Gini']:.4f}")
print(f"\nThreshold óptimo: {modelo_export['threshold_optimo']:.3f}")
print(f"\nContenido del archivo:")
print(f"  • modelo: DummyClassifier optimizado")
print(f"  • preprocessor: ColumnTransformer (StandardScaler + OneHotEncoder)")
print(f"  • features_seleccionadas: {final_selected_features}")
print(f"  • threshold_optimo: {modelo_export['threshold_optimo']:.3f}")
print(f"  • metricas: diccionario con todas las métricas")
print(f"  • mejores_parametros: {modelo_export['mejores_parametros']}")

MODELO EXPORTADO EXITOSAMENTE

Ruta: ../models/clasificacion_adicionales.pkl

Modelo: DummyClassifier
Mejores parámetros: {'strategy': 'stratified'}

Métricas en Test Set:
  F1-Score:  0.0706
  Accuracy:  0.9012
  Precision: 0.0690
  Recall:    0.0723
  AUC-ROC:   0.5094
  Gini:      0.0189

Threshold óptimo: 0.100

Contenido del archivo:
  • modelo: DummyClassifier optimizado
  • preprocessor: ColumnTransformer (StandardScaler + OneHotEncoder)
  • features_seleccionadas: ['2_o_mas_inquilinos_Si', 'estudios_area_Ciencias', 'estudios_area_Humanidades', 'estudios_area_Otro', 'extintor_incendios_Si']
  • threshold_optimo: 0.100
  • metricas: diccionario con todas las métricas
  • mejores_parametros: {'strategy': 'stratified'}
